<a href="https://colab.research.google.com/github/AdiKaipa/Session-9-Assignment-/blob/main/Language_Representation_using_Encoder_Decoder_on_AmbigQA_(Model_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AmbigQA, a new open-domain question answering task that consists of predicting a set of question and answer pairs, where each plausible answer is associated with a disambiguated rewriting of the original question. A data set covering 14,042 open-ended QI-open questions.

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd
import json
path = '/content/drive/MyDrive/END/END 8/train.json'

In [3]:
df = pd.read_json(path, lines=False, orient=str)
df.shape

(10036, 7)

In [4]:
df.head()

,viewed_doc_titles,used_queries,annotations,nq_answer,id,nq_doc_title,question
0,[The Simpsons],[{'query': 'When did the simpsons first air on...,"[{'type': 'multipleQAs', 'qaPairs': [{'questio...","[December 17 , 1989]",-4469503464110108160,The Simpsons,When did the simpsons first air on television?
1,[John Adams (miniseries)],"[{'query': 'John adams tv', 'results': [{'titl...","[{'type': 'singleAnswer', 'answer': ['David Mo...",[David Morse],4790842463458965504,John Adams (miniseries),Who played george washington in the john adams...
2,[Marriage age in the United States],"[{'query': 'legal age of marriage in usa', 're...","[{'type': 'multipleQAs', 'qaPairs': [{'questio...","[18, Nebraska ( 19 ), Mississippi ( 21 )]",-6631915997977101312,Age of marriage in the United States,What is the legal age of marriage in usa?
3,"[Barefoot in the Park, Barefoot in the Park (f...",[{'query': 'Who starred in barefoot in the par...,"[{'type': 'multipleQAs', 'qaPairs': [{'questio...","[Robert Redford, Elizabeth Ashley]",-3098213414945179648,Barefoot in the Park,Who starred in barefoot in the park on broadway?
4,"[Timeline of the Manhattan Project, Manhattan ...",[{'query': 'When did the manhattan project beg...,"[{'type': 'multipleQAs', 'qaPairs': [{'questio...",[From 1942 to 1946],-927805218867163520,Timeline of the Manhattan Project,When did the manhattan project began and end?


In [5]:
df = df[['question', 'nq_answer']]

In [6]:
df.head()

,question,nq_answer
0,When did the simpsons first air on television?,"[December 17 , 1989]"
1,Who played george washington in the john adams...,[David Morse]
2,What is the legal age of marriage in usa?,"[18, Nebraska ( 19 ), Mississippi ( 21 )]"
3,Who starred in barefoot in the park on broadway?,"[Robert Redford, Elizabeth Ashley]"
4,When did the manhattan project began and end?,[From 1942 to 1946]


In [7]:
Ans = [str(s).strip('[]') for s in df['nq_answer']]
df['nq_answer'] = Ans
df.head()

,question,nq_answer
0,When did the simpsons first air on television?,"'December 17 , 1989'"
1,Who played george washington in the john adams...,'David Morse'
2,What is the legal age of marriage in usa?,"'18', 'Nebraska ( 19 )', 'Mississippi ( 21 )'"
3,Who starred in barefoot in the park on broadway?,"'Robert Redford', 'Elizabeth Ashley'"
4,When did the manhattan project began and end?,'From 1942 to 1946'


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext import data


from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [9]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [10]:
SRC = Field(tokenize = 'spacy', 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)
TRG = Field(tokenize = 'spacy', 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [11]:
fields = [('question', SRC),('nq_answer',TRG)]

In [12]:
example = [data.Example.fromlist([df.question[i],df.nq_answer[i]], fields) for i in range(df.shape[0])] 

In [13]:
QADataset = data.Dataset(example, fields)

In [14]:
vars(QADataset.examples[0])

{'nq_answer': ["'", 'december', '17', ',', '1989', "'"],
 'question': ['when',
  'did',
  'the',
  'simpsons',
  'first',
  'air',
  'on',
  'television',
  '?']}

In [15]:
(train, valid) = QADataset.split(split_ratio=[0.90, 0.15], random_state=random.seed(SEED))

In [16]:
(len(train), len(valid))

(8602, 1434)

In [17]:
SRC.build_vocab(train, min_freq = 2)
TRG.build_vocab(train, min_freq = 2)

In [18]:
print("size of SRC vocab: ", len(SRC.vocab))
print('size of TRG vocab: ', len(TRG.vocab))

size of SRC vocab:  4278
size of TRG vocab:  2837


In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
BATCH_SIZE = 128
train_iterator, valid_iterator = BucketIterator.splits((train, valid),sort = False, batch_size = BATCH_SIZE, device = device)

In [21]:
# Encoder
class Encoder(nn.Module):
  def __init__(self, input_dim, emb_dim, hid_dim, dropout):
    super().__init__()
    self.hid_dim = hid_dim
    self.embedding = nn.Embedding(input_dim, emb_dim)
    self.rnn = nn.GRU(emb_dim, hid_dim)
    self.dropout = nn.Dropout(dropout)
  def forward(self, src):
    embedded = self.dropout(self.embedding(src))

    outputs, hidden = self.rnn(embedded)

       
    #outputs = [src len, batch size, hid dim * n directions]
    #hidden = [n layers * n directions, batch size, hid dim]
    return hidden

In [22]:
# Decoder 
class Decoder(nn.Module):
  def __init__(self, output_dim, emb_dim, hid_dim, dropout):
    super().__init__()
    self.hid_dim = hid_dim,
    self.output_dim = output_dim
    self.embedding = nn.Embedding(output_dim, emb_dim)
    self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)
    self.fc = nn.Linear(emb_dim + hid_dim * 2, output_dim)
    self.dropout = nn.Dropout(dropout)

  def forward(self, input, hidden, context):
    #input = [batch size]
    #hidden = [n layers * n directions, batch size, hid dim]
    #context = [n layers * n directions, batch size, hid dim]
        
    #n layers and n directions in the decoder will both always be 1, therefore:
    #hidden = [1, batch size, hid dim]
    #context = [1, batch size, hid dim]
    input = input.unsqueeze(0)
    embedded = self.dropout(self.embedding(input))

    emb_con = torch.cat((embedded, context), dim = 2)
    output, hidden = self.rnn(emb_con, hidden)
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        
        #seq len, n layers and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
    output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), dim = 1)
    prediction = self.fc(output)
    
    return prediction, hidden
    


In [23]:
class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, device):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.device = device

    #assert encoder.hid_dim == decoder.hid_dim, \
    #  "Hidden dimensions of encoder and decoder must be equal"
  def forward(self, src, trg, teacher_forcing_ratio = 0.5):

    #src = [src len, batch size]
    #trg = [trg len, batch size]
    #teacher_forcing_ratio is probability to use teacher forcing
    #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time

    batch_size = src.shape[1]
    trg_len = trg.shape[0]
    trg_vocab_size = self.decoder.output_dim
    outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
    context = self.encoder(src)
    hidden = context
    input = trg[0, :]
    for t in range(1, trg_len):
      output, hidden = self.decoder(input, hidden, context)
      outputs[t] = output
      teacher_force = random.random() < teacher_forcing_ratio
      top1 = output.argmax(1)
      input = trg[t] if teacher_force else top1
    
    return outputs


In [24]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [25]:
def init_weights(m):
  for name, param in m.named_parameters():
    nn.init.normal_(param.data, mean = 0.0, std = 0.01)
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(4278, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(2837, 256)
    (rnn): GRU(768, 512)
    (fc): Linear(in_features=1280, out_features=2837, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [26]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 8,607,509 trainable parameters


In [27]:
# optimizer
optimizer = optim.Adam(model.parameters())

In [28]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [29]:
def train(model, iterator, optimizer, criterion, clip):
  model.train()
  epoch_loss = 0
  for i, batch in enumerate(iterator):
    src = batch.question
    trg = batch.nq_answer
    optimizer.zero_grad()
    output = model(src, trg)
    output_dim = output.shape[-1]
    output = output[1:].view(-1, output_dim)
    trg = trg[1:].view(-1)
    loss = criterion(output, trg)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    epoch_loss += loss.item()
  return epoch_loss / len(iterator)

In [30]:
def evaluate(model, iterator, criterion):
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.question
            trg = batch.nq_answer

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [31]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [32]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 8s
	Train Loss: 4.139 | Train PPL:  62.752
	 Val. Loss: 3.036 |  Val. PPL:  20.818
Epoch: 02 | Time: 0m 7s
	Train Loss: 3.226 | Train PPL:  25.190
	 Val. Loss: 2.958 |  Val. PPL:  19.265
Epoch: 03 | Time: 0m 8s
	Train Loss: 3.110 | Train PPL:  22.412
	 Val. Loss: 2.971 |  Val. PPL:  19.513
Epoch: 04 | Time: 0m 8s
	Train Loss: 2.994 | Train PPL:  19.958
	 Val. Loss: 2.908 |  Val. PPL:  18.324
Epoch: 05 | Time: 0m 8s
	Train Loss: 2.920 | Train PPL:  18.543
	 Val. Loss: 2.832 |  Val. PPL:  16.984
Epoch: 06 | Time: 0m 8s
	Train Loss: 2.727 | Train PPL:  15.283
	 Val. Loss: 2.872 |  Val. PPL:  17.675
Epoch: 07 | Time: 0m 8s
	Train Loss: 2.663 | Train PPL:  14.334
	 Val. Loss: 2.742 |  Val. PPL:  15.512
Epoch: 08 | Time: 0m 8s
	Train Loss: 2.575 | Train PPL:  13.137
	 Val. Loss: 2.771 |  Val. PPL:  15.973
Epoch: 09 | Time: 0m 8s
	Train Loss: 2.503 | Train PPL:  12.223
	 Val. Loss: 2.721 |  Val. PPL:  15.193
Epoch: 10 | Time: 0m 8s
	Train Loss: 2.419 | Train PPL:  11.238
